# load toy dataset

In [1]:
import numpy as np
import torch
# create train_set , validation_set from merged npy array
arr = torch.from_numpy(np.load('merged_array.npy'))
# torch.Size([9494, 128, 128]) -> torch.Size([128, 128, 9494])
print(arr.shape,arr[0][0].shape)
arr = arr.permute(1, 2, 0)
print(arr.shape,arr[0][0].shape)
arr = arr.flatten(start_dim=0, end_dim=1)
print(arr.shape)
print(arr.unsqueeze(0).shape)
#arr = arr.permute(1, 0)
arr = arr.unsqueeze(0)

torch.Size([202, 6016, 128]) torch.Size([128])
torch.Size([6016, 128, 202]) torch.Size([202])
torch.Size([770048, 202])
torch.Size([1, 770048, 202])


# load 1d unet / diffusion model from gh repo, create example train_sequence and train. create diffusion bases sample version 

In [8]:
import torch
from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D, Trainer1D 
# import torch dataset
from torch.utils.data import Dataset

model = Unet1D(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    channels = 202
)

diffusion = GaussianDiffusion1D(
    model,
    seq_length = 32,
    timesteps = 100,
    objective = 'pred_v'
)

training_seq = torch.rand(1, 202, 32) # features are normalized from 0 to 1 # b, ch, seq_length
#training_seq = arr

trainer = Trainer1D(
    diffusion,
    dataset = training_seq,
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 100,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
)
trainer.train()

# after a lot of training

sampled_seq = diffusion.sample(batch_size = 4)
sampled_seq.shape

  0%|          | 0/100 [00:00<?, ?it/s]

training complete


sampling loop time step:   0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([4, 202, 32])

In [10]:
model

Unet1D(
  (init_conv): Conv1d(202, 64, kernel_size=(7,), stride=(1,), padding=(3,))
  (time_mlp): Sequential(
    (0): SinusoidalPosEmb()
    (1): Linear(in_features=64, out_features=256, bias=True)
    (2): GELU(approximate='none')
    (3): Linear(in_features=256, out_features=256, bias=True)
  )
  (downs): ModuleList(
    (0): ModuleList(
      (0-1): 2 x ResnetBlock(
        (mlp): Sequential(
          (0): SiLU()
          (1): Linear(in_features=256, out_features=128, bias=True)
        )
        (block1): Block(
          (proj): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
          (norm): GroupNorm(8, 64, eps=1e-05, affine=True)
          (act): SiLU()
        )
        (block2): Block(
          (proj): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
          (norm): GroupNorm(8, 64, eps=1e-05, affine=True)
          (act): SiLU()
        )
        (res_conv): Identity()
      )
      (2): Residual(
        (fn): PreNorm(
          (fn): LinearAtt

In [13]:
sampled_seq.shape

torch.Size([4, 202, 32])

In [19]:
trainer = Trainer1D(
    diffusion,
    dataset = arr[:,:,:64],
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 100,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
)
trainer.train()

# after a lot of training

sampled_seq = diffusion.sample(batch_size = 4)
sampled_seq.shape 

  0%|          | 0/100 [00:00<?, ?it/s]

AssertionError: seq length must be 32